## Accurate Fourier derivatives, to include the full period (i.e. f(a) = f(b)) or to include the interval [a,b)?

In [43]:
import numpy as np
import numpy.fft 

def func(x):
    return np.sin(2*x)

def dfunc(x):
    return 2*np.cos(2*x)

In [31]:
import numpy as np
import numpy.fft 

def func(x):
    return x ** 3

def dfunc(x):
    return 3 * x ** 2

In [80]:
Nx = 20
ax, bx = 0, np.pi
L = bx - ax
x = np.linspace(0, np.pi, Nx)
f = func(x)
df = dfunc(x)

Ff = np.fft.fft(f)
wave_numbers = np.zeros_like(x)

wave_index = np.arange(Nx)
wave_numbers = np.where(wave_index <= Nx / 2, 2*np.pi*wave_index / L, 2*np.pi*(wave_index - Nx) / L)

print wave_numbers 

dFf = 1j * wave_numbers * Ff

df = np.real(np.fft.ifft(dFf))

[  0.   2.   4.   6.   8.  10.  12.  14.  16.  18.  20. -18. -16. -14. -12.
 -10.  -8.  -6.  -4.  -2.]


In [81]:
x_active = x[:-1]
Nx_active = Nx-1

f2 = func(x_active)
df2 = dfunc(x_active)

Ff2 = np.fft.fft(f2)

wave_numbers2 = np.zeros_like(x_active)

wave_index2 = np.arange(Nx_active)
wave_numbers2 = np.where(wave_index2 <= Nx_active / 2, 2*np.pi*wave_index2 / L, 2*np.pi*(wave_index2 - Nx_active) / L)
print wave_numbers2

dFf = 1j * wave_numbers2 * Ff2

df2 = np.real(np.fft.ifft(dFf))

[  0.   2.   4.   6.   8.  10.  12.  14.  16.  18. -18. -16. -14. -12. -10.
  -8.  -6.  -4.  -2.]


In [82]:
%matplotlib

plt.stem(wave_numbers, np.abs(Ff))
plt.stem(wave_numbers2, np.abs(Ff2), linefmt = 'r-', markerfmt = 'ro')

Using matplotlib backend: TkAgg


<Container object of 3 artists>

In [83]:
plt.stem?

In [84]:
%matplotlib

import matplotlib.pyplot as plt

f_exact = func(x)
df_exact = dfunc(x)

plt.plot(x,f_exact)
plt.plot(x,df_exact, linewidth = 3, label = 'exact')
plt.plot(x,df, 'o', label = 'full period')
plt.plot(x_active,df2, 'D', label = 'open period')
plt.legend(loc = 'best')

Using matplotlib backend: TkAgg


In [78]:
print df
print df2

[ 0.6481748   2.62800677  1.27365945  1.41787217  0.32426737 -0.03286689
 -0.94704695 -1.35724916 -1.89131626 -2.0635013  -2.0635013  -1.89131626
 -1.35724916 -0.94704695 -0.03286689  0.32426737  1.41787217  1.27365945
  2.62800677  0.6481748 ]
[ 2.          1.89163448  1.57828102  1.09389632  0.49097097 -0.16515869
 -0.80339085 -1.35456314 -1.7589475  -1.97272261 -1.97272261 -1.7589475
 -1.35456314 -0.80339085 -0.16515869  0.49097097  1.09389632  1.57828102
  1.89163448]


In [79]:
print f
print f2

[  0.00000000e+00   3.24699469e-01   6.14212713e-01   8.37166478e-01
   9.69400266e-01   9.96584493e-01   9.15773327e-01   7.35723911e-01
   4.75947393e-01   1.64594590e-01  -1.64594590e-01  -4.75947393e-01
  -7.35723911e-01  -9.15773327e-01  -9.96584493e-01  -9.69400266e-01
  -8.37166478e-01  -6.14212713e-01  -3.24699469e-01  -2.44929360e-16]
[ 0.          0.32469947  0.61421271  0.83716648  0.96940027  0.99658449
  0.91577333  0.73572391  0.47594739  0.16459459 -0.16459459 -0.47594739
 -0.73572391 -0.91577333 -0.99658449 -0.96940027 -0.83716648 -0.61421271
 -0.32469947]


In [2]:
%load_ext Cython

In [3]:
%%cython --a
import numpy as np
cimport numpy as np
import cython
from cpython cimport bool

@cython.boundscheck(False)
@cython.wraparound(False)
def sort_points(points):
    cdef list pages
    cdef dict page
    cdef bool dashed
    cdef int default_color, color, x, y, z
    cdef tuple spec
    cdef np.ndarray point

    pages = []
    page = {(False, 15000): []}
    dashed = False
    default_color = 15000
    color = 15000
    spec = (dashed, color)
    for point in points:
        x, y, z = point
        spec = (dashed, color)
        if spec not in page:
            page[spec] = []

        if z == 2:
            page[spec].append((x, y))
        elif z == 3:
            page[spec].append((np.NaN, np.NaN))
            page[spec].append((x, y))
        elif z == 997:
            dashed = y == 6
        elif z == 900:
            color = x
        elif z == 9999:
            pages.append(page)
            page = {}
            dashed = False
            color = default_color

    return pages

/usr/local/lib/python2.7/dist-packages/pygments/plugin.py:39: UserWarning: Module dap was already imported from None, but /usr/lib/python2.7/dist-packages is being added to sys.path
  import pkg_resources
